In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor as gb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn import preprocessing 
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from sklearn import preprocessing
train = pd.read_csv("drive/My Drive/Colab Notebooks/CMPSC 448/xgboost/train_data.csv")

train = train.drop("key", axis = 1)

X, y = train.iloc[:,:-1],train.iloc[:,-1]

for i in range(len(y)):
    if y[i] == "Class_1":
        y[i] = 0
    if y[i] == "Class_2":
        y[i] = 1
    if y[i] == "Class_3":
        y[i] = 2
    if y[i] == "Class_4":
        y[i] = 3
    if y[i] == "Class_5":
        y[i] = 4
    if y[i] == "Class_6":
        y[i] = 5
    if y[i] == "Class_7":
        y[i] = 6
    if y[i] == "Class_8":
        y[i] = 7
    if y[i] == "Class_9":
        y[i] = 8
X = preprocessing.MaxAbsScaler().fit_transform(X)
pca = PCA(n_components=62, svd_solver = 'randomized')
X = pca.fit(X).transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify = y)
eval_set = [(X_test, y_test)]
y_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarni

60455    5
2097     8
15765    8
5914     1
60362    2
        ..
31172    5
27784    6
15866    4
2758     2
11173    5
Name: category, Length: 18564, dtype: object

In [5]:
from sklearn.calibration import CalibratedClassifierCV

xgb_clf = xgb.XGBClassifier(objective = "multi:softmax", max_depth=8, nfold = 5,nrounds = 10,n_estimators=525,min_child_weight=3, sample_type='weighted',colsample_bytree=0.7,subsample=0.7,eta=0.1,seed=4)
calibrated_xgb = CalibratedClassifierCV(xgb, cv = 5, method = 'isotonic')
xgb_fit = xgb_clf.fit(X_train, y_train, eval_set = eval_set, eval_metric = 'mlogloss', early_stopping_rounds =15)


[0]	validation_0-mlogloss:1.96643
Will train until validation_0-mlogloss hasn't improved in 15 rounds.
[1]	validation_0-mlogloss:1.80405
[2]	validation_0-mlogloss:1.67389
[3]	validation_0-mlogloss:1.57365
[4]	validation_0-mlogloss:1.48445
[5]	validation_0-mlogloss:1.40409
[6]	validation_0-mlogloss:1.33556
[7]	validation_0-mlogloss:1.27433
[8]	validation_0-mlogloss:1.21979
[9]	validation_0-mlogloss:1.17362
[10]	validation_0-mlogloss:1.13073
[11]	validation_0-mlogloss:1.09169
[12]	validation_0-mlogloss:1.05591
[13]	validation_0-mlogloss:1.024
[14]	validation_0-mlogloss:0.994148
[15]	validation_0-mlogloss:0.968005
[16]	validation_0-mlogloss:0.943074
[17]	validation_0-mlogloss:0.919266
[18]	validation_0-mlogloss:0.897732
[19]	validation_0-mlogloss:0.878651
[20]	validation_0-mlogloss:0.861334
[21]	validation_0-mlogloss:0.845338
[22]	validation_0-mlogloss:0.82991
[23]	validation_0-mlogloss:0.815585
[24]	validation_0-mlogloss:0.802145
[25]	validation_0-mlogloss:0.789681
[26]	validation_0-mlog

In [0]:
from sklearn.metrics import log_loss
preds_y = xgb_fit.predict_proba(X_test)
log_loss(y_test, preds_y)

ValueError: ignored

In [0]:
selection = SelectFromModel(xgb_fit, threshold = thresholds[0], prefit = True)
select_X_train = selection.transform(X_train)
select_X_test = selection.transform(X_test)
selection_model = xgb.XGBClassifier(objective = "multi:softmax", max_depth=250, tree_method = 'hist', grow_policy='lossguide', normalize_type = 'forest', csv_weights=10, num_round = 1200, nfold = 5,nrounds = 10,n_estimators=15000,min_child_weight=35, sample_type='weighted',colsample_bytree=0.7,subsample=0.8,eta=0.0001,seed=4)
eval_set = [(select_X_test, y_test)]
selection_model.fit(select_X_train, y_train, eval_set = eval_set, eval_metric="mlogloss", early_stopping_rounds=15)

[0]	validation_0-mlogloss:1.98602
Will train until validation_0-mlogloss hasn't improved in 15 rounds.
[1]	validation_0-mlogloss:1.82317
[2]	validation_0-mlogloss:1.69254
[3]	validation_0-mlogloss:1.5876
[4]	validation_0-mlogloss:1.49544
[5]	validation_0-mlogloss:1.41734
[6]	validation_0-mlogloss:1.3443
[7]	validation_0-mlogloss:1.28422
[8]	validation_0-mlogloss:1.22993
[9]	validation_0-mlogloss:1.17837
[10]	validation_0-mlogloss:1.13369
[11]	validation_0-mlogloss:1.09253
[12]	validation_0-mlogloss:1.05558
[13]	validation_0-mlogloss:1.02033
[14]	validation_0-mlogloss:0.990855
[15]	validation_0-mlogloss:0.962167
[16]	validation_0-mlogloss:0.934676
[17]	validation_0-mlogloss:0.910086
[18]	validation_0-mlogloss:0.887145
[19]	validation_0-mlogloss:0.866125
[20]	validation_0-mlogloss:0.847083
[21]	validation_0-mlogloss:0.830098
[22]	validation_0-mlogloss:0.814776
[23]	validation_0-mlogloss:0.799747
[24]	validation_0-mlogloss:0.784916
[25]	validation_0-mlogloss:0.772186
[26]	validation_0-mlo

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, csv_weights=10,
              eta=0.0001, gamma=0, grow_policy='lossguide', learning_rate=0.1,
              max_delta_step=0, max_depth=250, min_child_weight=35,
              missing=None, n_estimators=15000, n_jobs=1, nfold=5,
              normalize_type='forest', nrounds=10, nthread=None, num_round=1200,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, sample_type='weighted', scale_pos_weight=1, seed=4,
              silent=None, subsample=0.8, ...)

ValueError: ignored

In [9]:
test = pd.read_csv("drive/My Drive/Colab Notebooks/CMPSC 448/xgboost/test_data.csv")
test = test.drop("key", axis = 1)
test = preprocessing.MaxAbsScaler().fit(test).transform(test)
pca = PCA(n_components=62, svd_solver = 'randomized')
test = pca.fit_transform(test)
preds = xgb_fit.predict_proba(test)
preds

array([[0.00287864, 0.35138133, 0.2390139 , ..., 0.05416419, 0.05013962,
        0.00273514],
       [0.02323537, 0.24189124, 0.08008125, ..., 0.226524  , 0.03441353,
        0.01871321],
       [0.0188963 , 0.22859316, 0.04767476, ..., 0.01358171, 0.33679667,
        0.00199734],
       ...,
       [0.03348554, 0.05027708, 0.02428818, ..., 0.02806295, 0.03860673,
        0.3907879 ],
       [0.04415145, 0.11020876, 0.22452424, ..., 0.04827682, 0.04293514,
        0.05457564],
       [0.03951892, 0.02720563, 0.01476785, ..., 0.05801875, 0.0381388 ,
        0.1118891 ]], dtype=float32)

In [10]:
samp_sub = pd.read_csv("drive/My Drive/Colab Notebooks/CMPSC 448/xgboost/sample_submission.csv")
output = pd.DataFrame(preds,columns=["group_0"+str(i) for i in range(1,10)])
output.insert(0, 'key', samp_sub["key"])
output

,key,group_01,group_02,group_03,group_04,group_05,group_06,group_07,group_08,group_09
0,45451,0.002879,0.351381,0.239014,0.086883,0.000452,0.212351,0.054164,0.050140,0.002735
1,63656,0.023235,0.241891,0.080081,0.046891,0.000401,0.327849,0.226524,0.034414,0.018713
2,74635,0.018896,0.228593,0.047675,0.004684,0.001497,0.346279,0.013582,0.336797,0.001997
3,89761,0.001687,0.203940,0.071814,0.364575,0.000394,0.243824,0.098798,0.014046,0.000923
4,3559,0.065807,0.039091,0.015305,0.020150,0.001583,0.280763,0.003685,0.056975,0.516641
...,...,...,...,...,...,...,...,...,...,...
144363,28524,0.033291,0.069993,0.041033,0.087264,0.002654,0.668421,0.014742,0.011912,0.070690
144364,135857,0.040434,0.105701,0.113721,0.227814,0.001085,0.274725,0.045137,0.108317,0.083066
144365,76667,0.033486,0.050277,0.024288,0.196552,0.000244,0.237696,0.028063,0.038607,0.390788
144366,93453,0.044151,0.110209,0.224524,0.176051,0.000695,0.298582,0.048277,0.042935,0.054576


In [0]:
output.to_csv("doubtful.csv", index = False)
!cp doubtful.csv "drive/My Drive/Colab Notebooks/CMPSC 448/xgboost/"